In [1]:
import pandas as pd

In [2]:
# max row 출력
pd.options.display.max_rows = 60

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/원본 데이터/train.csv')

train & test 데이터 합치기

In [4]:
# test = pd.read_csv('/content/drive/MyDrive/ch5. 소득예측/원본 데이터/test.csv')

In [5]:
# concat_df = pd.concat([train, test], axis = 0)

In [6]:
# df = concat_df.copy()

- hispanic_origin 공백제거 및 결측치 처리

In [7]:
# 공백제거
df['Hispanic_Origin'] = df['Hispanic_Origin'].str.strip()

In [8]:
#check
df[df['Hispanic_Origin'] == 'NA'].head(1)

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
629,TRAIN_00629,37,F,Associates degree (Vocational),Children or Armed Forces,0,Agriculture,Farming & Forestry & Fishing,White,NA,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Unknown,0


In [9]:
# his 결측치 처리 x
df_hx = df.copy()

In [10]:
# his 결측치 처리 o
df_ho = df.copy()

In [11]:
# Do not know
his_con1 = df_ho['Hispanic_Origin'] == 'Do not know'
df_ho.loc[his_con1,'Hispanic_Origin'] = 'NA'


# All other 경우
for i in ['US', 'Germany','Philippines','Unknown','Canada','Italy','Jamaica'] :
    his_con1 = df_ho['Hispanic_Origin'] == 'NA'
    his_con2 = df_ho['Birth_Country'] == i
    df_ho.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'All other'


# Mexico
his_con1 = df_ho['Hispanic_Origin'] == 'NA'
his_con2 = df_ho['Birth_Country'] == 'Mexico'
df_ho.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'Mexican (Mexicano)'

# Cuba
his_con2 = df_ho['Birth_Country'] == 'Cuba'
df_ho.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'Cuban'

# Nicaragua
his_con2 = df_ho['Birth_Country'] == 'Nicaragua'
df_ho.loc[his_con1 & his_con2,'Hispanic_Origin'] = 'Central or South American'

여기서 차이점 발견 :    
do not know를 NA로 바꾸고 처리하지 않고    
do not know를 All other로 바꿈

In [12]:
# check
df_ho[df_ho['Hispanic_Origin'] == "NA"]

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income


In [13]:
df_hx.drop(["Household_Status"],axis = 1, inplace = True)
df_ho.drop(["Household_Status"],axis = 1, inplace = True)

- birth_country 결측치 처리

In [14]:
df_hobo = df_ho.copy()
df_hobx = df_ho.copy()
df_hxbo = df_hx.copy()
df_hxbx = df_hx.copy()

- H O B O

In [15]:
# df_hobo df

birth_con1 = df_hobo['Birth_Country'] == 'Unknown'

# 1) 부모 국적이 채워진 경우
# -> 아빠의 국적만 있는 경우
birth_con2 = (df_hobo['Birth_Country (Father)'] != 'Unknown') & (df_hobo['Birth_Country (Mother)'] == 'Unknown')
df_hobo.loc[(birth_con1) & (birth_con2),'Birth_Country'] = df_hobo['Birth_Country (Father)']

# -> 엄마의 국적만 채워진 경우
birth_con3 = (df_hobo['Birth_Country (Father)'] == 'Unknown') & (df_hobo['Birth_Country (Mother)'] != 'Unknown')
df_hobo.loc[birth_con1 & birth_con3, 'Birth_Country'] = df_hobo['Birth_Country (Mother)']

# -> 둘다 채워진 경우
birth_con4 = (df_hobo['Birth_Country (Father)'] != 'Unknown') & (df_hobo['Birth_Country (Mother)'] != 'Unknown')
df_hobo.loc[birth_con1 & birth_con4, 'Birth_Country'] = df_hobo['Birth_Country (Father)']

In [16]:
# 2) 부모 국적 모두 Unkown인 경우
birth_con5 = (df_hobo['Birth_Country (Father)'] == 'Unknown') & (df_hobo['Birth_Country (Mother)'] == 'Unknown')

In [17]:
# Hispanic 별로 가장 많은 개수를 가진 나라 추출하기
a = list(df_hobo['Hispanic_Origin'].unique())
dic = {}
for i in  a:
    dic[i] = list(df_hobo[df_hobo['Hispanic_Origin'] == i].groupby('Birth_Country')['Income'].agg(['count']).sort_values(by = 'count', ascending = False).iloc[:1].index)

In [18]:
# 각 hispanic에 따라 출생지 입력하기
for i, j in dic.items():
  con1 = (df_hobo['Hispanic_Origin'] == i)
  df_hobo.loc[birth_con1 & birth_con5 & con1, 'Birth_Country'] = j[0]

In [19]:
# 전체적인 Check
df_hobo[df_hobo['Birth_Country'] == 'Unknown']['ID'].count()

0

- H : X B : O

In [20]:
# df_hxbo

birth_con1 = df_hxbo['Birth_Country'] == 'Unknown'

# 1) 부모 국적이 채워진 경우
# -> 아빠의 국적만 있는 경우
birth_con2 = (df_hxbo['Birth_Country (Father)'] != 'Unknown') & (df_hxbo['Birth_Country (Mother)'] == 'Unknown')
df_hxbo.loc[(birth_con1) & (birth_con2),'Birth_Country'] = df_hxbo['Birth_Country (Father)']

# -> 엄마의 국적만 채워진 경우
birth_con3 = (df_hxbo['Birth_Country (Father)'] == 'Unknown') & (df_hxbo['Birth_Country (Mother)'] != 'Unknown')
df_hxbo.loc[birth_con1 & birth_con3, 'Birth_Country'] = df_hxbo['Birth_Country (Mother)']

# -> 둘다 채워진 경우
birth_con4 = (df_hxbo['Birth_Country (Father)'] != 'Unknown') & (df_hxbo['Birth_Country (Mother)'] != 'Unknown')
df_hxbo.loc[birth_con1 & birth_con4, 'Birth_Country'] = df_hxbo['Birth_Country (Father)']

In [21]:
# 2) 부모 국적 모두 Unkown인 경우
birth_con5 = (df_hxbo['Birth_Country (Father)'] == 'Unknown') & (df_hxbo['Birth_Country (Mother)'] == 'Unknown')

In [22]:
# Hispanic 별로 가장 많은 개수를 가진 나라 추출하기
a = list(df_hxbo['Hispanic_Origin'].unique())
dic = {}
for i in  a:
    dic[i] = list(df_hxbo[df_hxbo['Hispanic_Origin'] == i].groupby('Birth_Country')['Income'].agg(['count']).sort_values(by = 'count', ascending = False).iloc[:1].index)

In [23]:
dic

{'All other': ['US'],
 'Cuban': ['Cuba'],
 'Mexican (Mexicano)': ['Mexico'],
 'Central or South American': ['US'],
 'Puerto Rican': ['US'],
 'Mexican-American': ['US'],
 'Other Spanish': ['US'],
 'Do not know': ['US'],
 'Chicano': ['US'],
 'NA': ['US']}

In [24]:
# 각 hispanic에 따라 출생지 입력하기
for i, j in dic.items():
  con1 = (df_hxbo['Hispanic_Origin'] == i)
  df_hxbo.loc[birth_con1 & birth_con5 & con1, 'Birth_Country'] = j[0]

In [25]:
# 전체적인 Check
df_hxbo[df_hxbo['Birth_Country'] == 'Unknown']['ID'].count()

0

In [26]:
df_hxbo[df_hxbo['Hispanic_Origin'] == 'NA']

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
629,TRAIN_00629,37,F,Associates degree (Vocational),Children or Armed Forces,0,Agriculture,Farming & Forestry & Fishing,White,NA,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Unknown,0
726,TRAIN_00726,40,M,High graduate,Full-Time,52,Manufacturing (Non-durable),Services,White,NA,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,0
1121,TRAIN_01121,38,M,High graduate,Children or Armed Forces,52,Business & Repair,Services,White,NA,...,Native,US,US,US,Single,0,0,0,Under Median,0
1748,TRAIN_01748,31,M,College,Children or Armed Forces,52,Manufacturing (Non-durable),Machine Operators & Inspectors,White,NA,...,Foreign-born (Non-US Citizen),Nicaragua,Nicaragua,Nicaragua,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,0
1962,TRAIN_01962,69,M,High Junior,Children or Armed Forces,0,Not in universe or children,Unknown,White,NA,...,Foreign-born (Non-US Citizen),Philippines,Philippines,Philippines,Nonfiler,0,0,0,Under Median,0
2322,TRAIN_02322,28,M,College,Children or Armed Forces,41,Business & Repair,Handlers/Cleaners,White,NA,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,0
2526,TRAIN_02526,45,M,High Freshman,Full-Time,52,Manufacturing (Durable),Machine Operators & Inspectors,White,NA,...,Native,US,US,US,Single,0,0,0,Under Median,890
2837,TRAIN_02837,26,F,College,Children or Armed Forces,52,Communications,Admin Support (include Clerical),White,NA,...,Native,US,US,US,Single,0,0,0,Under Median,1532
2853,TRAIN_02853,59,F,College,Children or Armed Forces,0,Not in universe or children,Unknown,White,NA,...,Foreign-born (Non-US Citizen),Germany,Germany,Germany,Married Filling Jointly both under 65 (MFJ),0,0,500,Under Median,0
3022,TRAIN_03022,36,F,College,Part-Time (Usually Full-Time),52,Medical (except Hospitals),Services,White,NA,...,Native,US,US,US,Head of Household (HOH),0,0,0,Under Median,786


- birth_country (father) & (mather) 칼럼 지우기

In [27]:
df_hobo.drop(["Birth_Country (Father)","Birth_Country (Mother)"], axis = 1, inplace = True)
df_hobx.drop(["Birth_Country (Father)","Birth_Country (Mother)"], axis = 1, inplace = True)
df_hxbo.drop(["Birth_Country (Father)","Birth_Country (Mother)"], axis = 1, inplace = True)
df_hxbx.drop(["Birth_Country (Father)","Birth_Country (Mother)"], axis = 1, inplace = True)

In [28]:
# # 수치형 변수 분포 확인하기
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import matplotlib.font_manager as fm

# plt.rcParams['axes.unicode_minus'] = False  # matplotlib 마이너스기호 표시
# plt.rc('font', family='NanumGothic')  # matplotlib 한글폰트 표시

# df_hobo.hist(bins=20, figsize=(20,20));

- Gains, Losses, Dividends 로그 변환

In [29]:
import numpy as np

df_hobo[["Gains","Losses","Dividends"]] = np.log1p(df_hobo[["Gains","Losses","Dividends"]])
df_hobx[["Gains","Losses","Dividends"]] = np.log1p(df_hobx[["Gains","Losses","Dividends"]])
df_hxbo[["Gains","Losses","Dividends"]] = np.log1p(df_hxbo[["Gains","Losses","Dividends"]])
df_hxbx[["Gains","Losses","Dividends"]] = np.log1p(df_hxbx[["Gains","Losses","Dividends"]])

In [30]:
df_hobo['Gains']

0        0.000000
1        0.000000
2        8.135054
3        0.000000
4        0.000000
           ...   
19995    0.000000
19996    0.000000
19997    0.000000
19998    0.000000
19999    0.000000
Name: Gains, Length: 20000, dtype: float64

- birth_country 기준으로 US O/X

In [31]:
# US O X 처리
def check_us(df) :
  cono = df['Birth_Country'] == 'US'
  conx = df['Birth_Country'] != 'US'
  df.loc[cono,'Us_ox'] = 1
  df.loc[conx,'Us_ox'] = 0

In [32]:
check_us(df_hobo)
check_us(df_hobx)
check_us(df_hxbo)
check_us(df_hxbx)

- Age_category 만들기

In [33]:
def make_age_category(df) :
  def category_age(age):
    age = (age//10)*10
    return age
  Age_category = df.Age.apply(category_age)
  df['Age_range'] = Age_category

In [34]:
make_age_category(df_hobo)
make_age_category(df_hobx)
make_age_category(df_hxbo)
make_age_category(df_hxbx)

- Gain과 Dividends 합 : Revenue

In [35]:
# 총 수입 칼럼만들기
def revenue_column(df) :
  df['Total_Gains'] = df['Gains'] + df['Dividends']

In [36]:
revenue_column(df_hobo)
revenue_column(df_hobx)
revenue_column(df_hxbo)
revenue_column(df_hxbx)

In [37]:
# profit 칼럼 만들기
def profit_column(df) :
  df['Net_Gains'] = df['Gains'] - df['Losses'] + df['Dividends']

In [38]:
profit_column(df_hobo)
profit_column(df_hobx)
profit_column(df_hxbo)
profit_column(df_hxbx)

- Working_age

In [39]:
# 20~60 경제활동 / 나머지
def active_age(age) :
  active_age = ''
  if age >= 20 and age <= 60  : active_age = 1
  else : active_age = 0
  return active_age

In [40]:
df_hobo['Age_workingage'] = df_hobo['Age'].apply(lambda x : active_age(x))
df_hobx['Age_workingage'] = df_hobx['Age'].apply(lambda x : active_age(x))
df_hxbo['Age_workingage'] = df_hxbo['Age'].apply(lambda x : active_age(x))
df_hxbx['Age_workingage'] = df_hxbx['Age'].apply(lambda x : active_age(x))

- 성별에 따른 연령대

In [41]:
df_hobo["Age_agegender"] = df_hobo["Age_range"].astype(str) + df_hobo["Gender"]
df_hobx["Age_agegender"] = df_hobx["Age_range"].astype(str) + df_hobo["Gender"]
df_hxbo["Age_agegender"] = df_hxbo["Age_range"].astype(str) + df_hobo["Gender"]
df_hxbx["Age_agegender"] = df_hxbx["Age_range"].astype(str) + df_hobo["Gender"]

- Education_univ 대학교 이상 / 나머지 구분

In [42]:

# edu_list[:1]
# edu_list[1:9]
# edu_list[9:10]
# edu_list[10:]

# Student /
def edu_univ(df):
  # 학력 순서대로 리스트 만들기
  edu_list = ['Children', 'Kindergarten','Elementary (1-4)', 'Elementary (5-6)', 'Middle (7-8)',
            'High Freshman', 'High Sophomore', 'High Junior', 'High Senior', 'High graduate',
            'Associates degree (Vocational)', 'Associates degree (Academic)', 'College', 'Bachelors degree',
            'Masters degree','Doctorate degree', 'Professional degree']
  for i in range(len(edu_list)):
    if i <= 9 :
      df.loc[(df['Education_Status'] == edu_list[i]), 'Education_univ'] = 0
    else :
      df.loc[(df['Education_Status'] == edu_list[i]), 'Education_univ'] = 1

In [43]:
edu_univ(df_hobo)
edu_univ(df_hobx)
edu_univ(df_hxbo)
edu_univ(df_hxbx)

- Children : 0
- Kindergarten ~ High Senior : 1
- High Graduate : 2
- Associate degree ~ : 3

In [44]:
# Child / Student / Graduate / Degree
def edu_stat(df):
  # 학력 순서대로 리스트 만들기
  edu_list = ['Children', 'Kindergarten','Elementary (1-4)', 'Elementary (5-6)', 'Middle (7-8)',
              'High Freshman', 'High Sophomore', 'High Junior', 'High Senior', 'High graduate',
              'Associates degree (Vocational)', 'Associates degree (Academic)', 'College', 'Bachelors degree',
              'Masters degree','Doctorate degree', 'Professional degree']
  for i in range(len(edu_list)):
    if i == 0 :
      df.loc[(df['Education_Status'] == edu_list[i]), 'Group_Education'] = 0
    elif i <= 8 :
      df.loc[(df['Education_Status'] == edu_list[i]), 'Group_Education'] = 1
    elif i == 9 :
      df.loc[(df['Education_Status'] == edu_list[i]), 'Group_Education'] = 2
    else:
      df.loc[(df['Education_Status'] == edu_list[i]), 'Group_Education'] = 3

In [45]:
edu_stat(df_hobo)
edu_stat(df_hobx)
edu_stat(df_hxbo)
edu_stat(df_hxbx)

- employment_group  3가지로 분류  
나머지 : 0    
파트타임 : 1   
풀타임 : 2    

In [46]:
def emp_group1(df) :
  # 1) 파트 / 풀 나머지m
  emp_list = ['Children or Armed Forces', 'Full-Time', 'Not Working', 'Choice Part-Time',
            'Part-Time (Usually Part-Time)', 'Part-Time (Usually Full-Time)','Seeking Full-Time', 'Seeking Part-Time']
  for i in range(len(emp_list)):
    if i == 0 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group1'] = 0
    elif i == 1 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group1'] = 2
    elif i == 2 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group1'] = 0
    elif 2 < i <= 5 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group1'] = 1
    elif i > 5:
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group1'] = 0

In [47]:
emp_group1(df_hobo)
emp_group1(df_hobx)
emp_group1(df_hxbo)
emp_group1(df_hxbx)

- employment_status : 5가지로 분류
- children or armed forces : 0
- part time : 1
- full time : 2
- not working : 3
- seeking : 4

In [48]:
def emp_group2(df) :
  # 2) 5가지로 분류
  emp_list = ['Children or Armed Forces', 'Full-Time', 'Not Working', 'Choice Part-Time',
            'Part-Time (Usually Part-Time)', 'Part-Time (Usually Full-Time)','Seeking Full-Time', 'Seeking Part-Time']
  for i in range(len(emp_list)):
    if i == 0 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group2'] = 0
    elif i == 1 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group2'] = 2
    elif i == 2 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group2'] = 3
    elif 2 < i <= 5 :
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group2'] = 1
    elif i > 5:
      df.loc[(df['Employment_Status'] == emp_list[i]), 'Employment_Group2'] = 4

In [49]:
emp_group2(df_hobo)
emp_group2(df_hobx)
emp_group2(df_hxbo)
emp_group2(df_hxbx)

- Martial_group
- 나머지 : 0
- single : 1
- Married : 2

In [50]:
def matial_group(df) :
  emp_list = ['Married',  'Married (Spouse Absent)', 'Married (Armed Force Spouse)', 'Single', 'Separated', 'Divorced', 'Widowed', ]
  # Child / Student / Graduate / Degree
  for i in range(len(emp_list)):
    if i <= 2 :
      df.loc[(df['Martial_Status'] == emp_list[i]), 'Martial_group'] = 2
    elif i == 3 :
      df.loc[(df['Martial_Status'] == emp_list[i]), 'Martial_group'] = 1
    else:
      df.loc[(df['Martial_Status'] == emp_list[i]), 'Martial_group'] = 0

In [51]:
matial_group(df_hobo)
matial_group(df_hobx)
matial_group(df_hxbo)
matial_group(df_hxbx)

- Native_yn
- Foreign : 0
- Native : 1

In [52]:
# Citizenship -> Native인지 아닌지 구분
def native_yn(df) :
  con1 = df['Citizenship'] == 'Native'
  con2 = df['Citizenship'] == 'Native (Born in Puerto Rico or US Outlying)'
  con3 = df['Citizenship'] == 'Native (Born Abroad)'
  con4 = df['Citizenship'] == 'Foreign-born (Non-US Citizen)'
  con5 = df['Citizenship'] == 'Foreign-born (Naturalized US Citizen)'
  df.loc[con1 | con2 | con3,'Native_yn'] = 1
  df.loc[con4 | con5,'Native_yn'] = 0

In [53]:
native_yn(df_hobo)
native_yn(df_hobx)
native_yn(df_hxbo)
native_yn(df_hxbx)

- Hispanic Origin
> All other 1 나머지 0

In [54]:
# Hispanic -> All other인지 아닌지 구분
def allother_yn(df) :
  con0 = df['Hispanic_Origin'] == 'All other'
  con1 = df['Hispanic_Origin'] != 'All other'
  df.loc[con0,'Allother_yn'] = 1
  df.loc[con1,'Allother_yn'] = 0

In [55]:
allother_yn(df_hobo)
allother_yn(df_hobx)
allother_yn(df_hxbo)
allother_yn(df_hxbx)

- Race 백인 1 나머지 0

In [56]:
# Race 인지 아닌
def Race_yn(df) :
  con0 = df['Race'] == 'White'
  con1 = df['Race'] != 'White'
  df.loc[con0,'Race_yn'] = 1
  df.loc[con1,'Race_yn'] = 2

In [57]:
Race_yn(df_hobo)
Race_yn(df_hobx)
Race_yn(df_hxbo)
Race_yn(df_hxbx)

In [58]:
df_hobo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     20000 non-null  object 
 1   Age                    20000 non-null  int64  
 2   Gender                 20000 non-null  object 
 3   Education_Status       20000 non-null  object 
 4   Employment_Status      20000 non-null  object 
 5   Working_Week (Yearly)  20000 non-null  int64  
 6   Industry_Status        20000 non-null  object 
 7   Occupation_Status      20000 non-null  object 
 8   Race                   20000 non-null  object 
 9   Hispanic_Origin        20000 non-null  object 
 10  Martial_Status         20000 non-null  object 
 11  Household_Summary      20000 non-null  object 
 12  Citizenship            20000 non-null  object 
 13  Birth_Country          20000 non-null  object 
 14  Tax_Status             20000 non-null  object 
 15  Ga

### Scaler 과정

- MinMax

In [59]:
df_hobomm = df_hobo.copy()
df_hobxmm = df_hobx.copy()
df_hxbomm = df_hxbo.copy()
df_hxbxmm = df_hxbx.copy()

In [60]:
# MinMax
def minmax(df):
  from sklearn.preprocessing import MinMaxScaler

  scaler = MinMaxScaler()
  df[["Age","Working_Week (Yearly)", "Gains","Losses","Dividends",
                "Total_Gains","Net_Gains"]] = scaler.fit_transform(df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]])

minmax(df_hobomm)
minmax(df_hobxmm)
minmax(df_hxbomm)
minmax(df_hxbxmm)

In [61]:
df_hobomm[['Gains','Total_Gains']].head(10)

,Gains,Total_Gains
0,0.000000,0.000000
1,0.000000,0.000000
2,0.706602,0.376644
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.293059
6,0.000000,0.000000
7,0.000000,0.353125
8,0.000000,0.000000
9,0.000000,0.000000


- standardScaler

In [62]:
df_hobosd = df_hobo.copy()
df_hobxsd = df_hobx.copy()
df_hxbosd = df_hxbo.copy()
df_hxbxsd = df_hxbx.copy()

In [63]:
def standard(df):
  from sklearn.preprocessing import StandardScaler

  scaler = StandardScaler()
  df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]] = scaler.fit_transform(df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]])

standard(df_hobosd)
standard(df_hobxsd)
standard(df_hxbosd)
standard(df_hxbxsd)

- Robust Scaler

In [64]:
df_hoborb = df_hobo.copy()
df_hobxrb = df_hobx.copy()
df_hxborb = df_hxbo.copy()
df_hxbxrb = df_hxbx.copy()

In [65]:
def robust(df) :
  from sklearn.preprocessing import RobustScaler

  scaler = RobustScaler()
  df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]] = scaler.fit_transform(df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]])

robust(df_hoborb)
robust(df_hobxrb)
robust(df_hxborb)
robust(df_hxbxrb)

- MaxAbs Scaler

In [66]:
df_hoboma = df_hobo.copy()
df_hobxma = df_hobx.copy()
df_hxboma = df_hxbo.copy()
df_hxbxma = df_hxbx.copy()

In [67]:
def maxabs(df) :
  from sklearn.preprocessing import MaxAbsScaler

  scaler = MaxAbsScaler()
  df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]] = scaler.fit_transform(df[["Age","Working_Week (Yearly)","Gains","Losses","Dividends","Total_Gains","Net_Gains"]])

maxabs(df_hoboma)
maxabs(df_hobxma)
maxabs(df_hxboma)
maxabs(df_hxbxma)

In [68]:
import sklearn as sk
print('Numpy Version :', sk.__version__)

Numpy Version : 1.4.0


In [69]:
# pip install --upgrade scikit-learn==1.4.0

In [70]:
df_hobo.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobo.csv', index = False)
df_hobx.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobx.csv', index = False)
df_hxbo.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbo.csv', index = False)
df_hxbx.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbx.csv', index = False)

In [71]:
df_hobomm.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobomm.csv', index = False)
df_hobxmm.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobxmm.csv', index = False)
df_hxbomm.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbomm.csv', index = False)
df_hxbxmm.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbxmm.csv', index = False)
df_hobosd.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobosd.csv', index = False)
df_hobxsd.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobxsd.csv', index = False)
df_hxbosd.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbosd.csv', index = False)
df_hxbxsd.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbxsd.csv', index = False)
df_hoborb.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hoborb.csv', index = False)
df_hobxrb.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobxrb.csv', index = False)
df_hxborb.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxborb.csv', index = False)
df_hxbxrb.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbxrb.csv', index = False)
df_hoboma.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hoboma.csv', index = False)
df_hobxma.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hobxma.csv', index = False)
df_hxboma.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxboma.csv', index = False)
df_hxbxma.to_csv('/content/drive/MyDrive/ch5. 소득예측/데이터 확인 과정/train_hxbxma.csv', index = False)